In [0]:
# =============================================================================
# CAMADA SILVER - TYPES - MAGIC: THE GATHERING
# =============================================================================
"""
Script Python para processamento da tabela TB_REF_SILVER_TYPES
Transformação e limpeza de dados da Bronze para Silver

USO DE SILVER_UTILS.PY:
- Centralização de funções comuns
- Padronização de processamento
- Redução de código duplicado
"""

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import logging
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *

# =============================================================================
# CARREGAMENTO DO MÓDULO UTILITÁRIO
# =============================================================================
# Importar funções do silver_utils usando %run (Databricks)
%run ./silver_utils

# =============================================================================
# CONFIGURAÇÃO INICIAL
# =============================================================================
def setup_logging():
    """Configura logging para o script"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

# =============================================================================
# CONFIGURAÇÃO
# =============================================================================

# Configuração manual 
config = create_manual_config("magic_the_gathering", get_secret("s3_bucket"))

# Setup Unity Catalog
setup_unity_catalog(config['catalog_name'], config['schema_silver'])



In [0]:
# =============================================================================
# PROCESSAMENTO USANDO SILVER_UTILS
# =============================================================================
# Criar processor
processor = SilverTableProcessor("TB_REF_SILVER_TYPES", config)

# Processar tabela de referência completa
df_silver = processor.process_reference_table(
    bronze_table_name="TB_BRONZE_TYPES",
    name_column="NME_TYPE",
    source_column="NME_SOURCE"
)

# =============================================================================
# VALIDAÇÃO E LOGS
# =============================================================================
if df_silver:
    print(f"Processamento concluído com sucesso!")
    print(f"Registros processados: {df_silver.count()}")
    print(f"Colunas finais: {df_silver.columns}")
else:
    print("Falha no processamento - DataFrame vazio") 